<a href="https://colab.research.google.com/github/mimrancomsats/Transformers/blob/main/mental_illness4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.7 MB/s 
     |████████████████████████████████| 6.6 MB 54.9 MB/s 
     |████████████████████████████████| 120 kB 76.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 62.7 MB/s 
     |████████████████████████████████| 115 kB 50.8 MB/s 
     |████████████████████████████████| 127 kB 65.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from datasets import load_dataset

In [3]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'posts_train.csv', 'test': 'posts_test.csv'}, encoding = "ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e6437d154e3a478d/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'post', 'class_name', 'class_id'],
        num_rows: 13727
    })
    test: Dataset({
        features: ['ID', 'post', 'class_name', 'class_id'],
        num_rows: 1488
    })
})

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [7]:
tokenizer("Attention is all you need")

{'input_ids': [101, 1335, 5208, 2116, 1110, 1155, 1128, 1444, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
def transform_labels(label):

    label = label['class_name']
    num = 0
    if label == 'none':
        num = 0
    elif label == 'anxiety':
        num = 1
    elif label == 'ptsd':
        num = 2
    elif label == 'adhd':
        num = 3
    elif label == 'bipolar':
        num = 4
    elif label == 'depression':
        num = 5

    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['post'], padding='max_length', truncation=True)

dataset = dataset.map(tokenize_data, batched=True)

remove_columns = ['ID', 'class_id']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/13727 [00:00<?, ?ex/s]

  0%|          | 0/1488 [00:00<?, ?ex/s]

In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)

In [10]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
train_dataset = dataset['train'].shuffle(seed=10).select(range(12727))
eval_dataset = dataset['train'].shuffle(seed=10).select(range(12727, 13727))

In [12]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: post, class_name. If post, class_name are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12727
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4773


Step,Training Loss
500,1.175500
1000,0.778300
1500,0.690100
2000,0.537600
2500,0.494700
3000,0.481400
3500,0.358900


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

Step,Training Loss
500,1.175500
1000,0.778300
1500,0.690100
2000,0.537600
2500,0.494700
3000,0.481400
3500,0.358900
4000,0.254700
4500,0.265400


Saving model checkpoint to test_trainer/checkpoint-4000
Configuration saved in test_trainer/checkpoint-4000/config.json
Model weights saved in test_trainer/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-4500
Configuration saved in test_trainer/checkpoint-4500/config.json
Model weights saved in test_trainer/checkpoint-4500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=4773, training_loss=0.5434945820013782, metrics={'train_runtime': 3746.1719, 'train_samples_per_second': 10.192, 'train_steps_per_second': 1.274, 'total_flos': 1.0046203994990592e+16, 'train_loss': 0.5434945820013782, 'epoch': 3.0})

In [13]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: post, class_name. If post, class_name are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 0.8489106893539429,
 'eval_runtime': 36.1499,
 'eval_samples_per_second': 27.663,
 'eval_steps_per_second': 3.458,
 'epoch': 3.0}